# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!pip install ale-py
!apt-get install -y swig
!pip install gymnasium[box2d]

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):

    def __init__(self, action_size, seed = 42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 8, stride = 4) # convolution
        self.bn1 = nn.BatchNorm2d(32) # batch normalization operation
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 4, stride = 2) # convolution
        self.bn2 = nn.BatchNorm2d(64) # batch normalization operation
        self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1) # convolution
        self.bn3 = nn.BatchNorm2d(64) # batch normalization operation
        self.conv4 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1) # convolution
        self.bn4 = nn.BatchNorm2d(128) # batch normalization operation
        self.fc1 = nn.Linear(10 * 10 * 128, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)

    def forward(self, state):
        x = F.relu(self.bn1(self.conv1(state)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import ale_py
import gymnasium as gym
env = gym.make('MsPacmanDeterministic-v0', full_action_space = False)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (210, 160, 3)
State size:  210
Number of actions:  9


/home/reefk/miniforge3/envs/ml-gpu-env/lib/python3.12/site-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99

### Preprocessing the frames

In [6]:
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame): # build tensor from frame
    frame = Image.fromarray(frame) # frame is now a PIL image object
    preprocess = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()]) # resize for neurel network, then convert PIL image to pytorch tensor
    return preprocess(frame).unsqueeze(0)

### Implementing the DCQN class

In [7]:
class Agent():

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    self.local_qnetwork = Network(action_size).to(self.device)
    self.target_qnetwork = Network(action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = deque(maxlen = 10000)

  def step(self, state, action, reward, next_state, done):
    state = preprocess_frame(state) # numpy array -> pytorch tensor
    next_state = preprocess_frame(next_state)
    self.memory.append((state, action, reward, next_state, done))
    if len(self.memory) > minibatch_size:
        experiences = random.sample(self.memory, k = minibatch_size)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = preprocess_frame(state).to(self.device) # input image -> input vector
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = zip(*experiences) # these are already pytorch tensors
    states = torch.from_numpy(np.vstack(
      states
      )).float().to(self.device)
    actions = torch.from_numpy(np.vstack(
      actions
      )).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(
      rewards
      )).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(
      next_states
      )).float().to(self.device)
    dones = torch.from_numpy(np.vstack(
      dones
      ).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1,actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

### Initializing the DCQN agent

In [8]:
agent = Agent(number_actions)
print(torch.cuda.get_device_name(agent))

Tesla P100-PCIE-16GB


### Training the DCQN agent

In [9]:
number_episodes = 2000
maximum_number_timestep_per_episode = 10000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timestep_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: 290.70
Episode 200	Average Score: 406.70
Episode 300	Average Score: 445.00
Episode 400	Average Score: 474.00
Episode 500	Average Score: 477.70
Episode 504	Average Score: 500.70
Environment solved in 404 episodes!	Average Score: 500.70


## Part 3 - Visualizing the results

In [21]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
import os

# Set the path to FFmpeg explicitly
os.environ["IMAGEIO_FFMPEG_EXE"] = "/home/reefk/miniforge3/envs/ml-gpu-env/bin/ffmpeg"

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [20]:
import os
os.system("ffmpeg -version")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
configuration: --prefix=/home/reefk/miniforge3/envs/ml-gpu-env --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646966651/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646966651/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646966651/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646966651/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-lib

0